In [153]:
import pandas as pd
func_tag_dic = {"Data Analysis":0, "Data Visualization":1 , "Data Preparation":2}

In [154]:
import re

In [155]:
description_corpura = pd.read_csv("training_corpus/extended_description.csv")
function_corpura = pd.read_csv("training_corpus/function.csv")

In [156]:
description_corpura

,Unnamed: 0,URL,contributor,excerpt
0,0,https://github.com/GoogleChrome/puppeteer,Allen Mao,Puppeteer is a Node library which provides a h...
1,1,https://github.com/JimmySuen/integral-human-pose,Allen Mao,The major contributors of this repository incl...
2,2,https://github.com/JimmySuen/integral-human-pose,Allen Mao,Integral Regression is initially described in ...
3,3,https://github.com/JimmySuen/integral-human-pose,Allen Mao,We build a 3D pose estimation system based mai...
4,4,https://github.com/JimmySuen/integral-human-pose,Allen Mao,The Integral Regression is also known as soft-...
...,...,...,...,...
409,409,https://github.com/deepmind/sonnet,Ling Li,Sonnet is a library built on top of TensorFlow...
410,410,https://github.com/deepmind/sonnet,Ling Li,It can be used to construct neural networks fo...
411,411,https://github.com/deepmind/sonnet,Ling Li,"(un/supervised learning, reinforcement learnin..."
412,412,https://github.com/deepmind/sonnet,Ling Li,Sonnet does not ship with a training framework...


In [157]:
description_corpura['label'] = None


In [158]:
func_dic = function_corpura.to_dict('index')
func_dic

{0: {'URL': 'https://github.com/KnowledgeCaptureAndDiscovery/somef/blob/master/experiments/training_corpus/repos/3D-ResNets-PyTorch-README.md',
  'contributor': 'Pratheek',
  'label': 'Data Preparation',
  'excerpt': 'If you want to classify your videos or extract video features of them using our pretrained models, use this code.\n'},
 1: {'URL': 'https://github.com/driving-behavior/DBNet',
  'contributor': 'Pratheek',
  'label': 'Data Preparation',
  'excerpt': "DBNet is a large-scale driving behavior dataset, which provides large-scale high-quality point clouds scanned by Velodyne lasers, high-resolution videos recorded by dashboard cameras and standard drivers' behaviors (vehicle speed, steering angle) collected by real-time sensors."},
 2: {'URL': 'https://github.com/hezhangsprinter/DCPDN',
  'contributor': 'Pratheek',
  'label': 'Data Preparation',
  'excerpt': 'We propose a new end-to-end single image dehazing method, called Densely Connected Pyramid Dehazing Network (DCPDN), whi

In [159]:
labels = []
count=0
drop = []
for index_dec,row_dec in description_corpura.iterrows():
    u = row_dec['URL'].split('/')
    for i in func_dic:
        if re.search(u[-1],func_dic[i]['URL']) is not None:
            #print(u[-1],func_dic[i]['URL'])
            description_corpura.loc[index_dec,'label'] = func_dic[i]['label']
    if description_corpura.loc[index_dec,'label'] is None:
        drop.append(index_dec)
dic = description_corpura.to_dict('index')  
url= []
for i in drop:
    url.append(dic[i]['URL'])
description_corpura= description_corpura.drop(drop,axis = 0,inplace = False)
description_corpura

,Unnamed: 0,URL,contributor,excerpt,label
0,0,https://github.com/GoogleChrome/puppeteer,Allen Mao,Puppeteer is a Node library which provides a h...,Data Preparation
1,1,https://github.com/JimmySuen/integral-human-pose,Allen Mao,The major contributors of this repository incl...,Data Preparation
2,2,https://github.com/JimmySuen/integral-human-pose,Allen Mao,Integral Regression is initially described in ...,Data Preparation
3,3,https://github.com/JimmySuen/integral-human-pose,Allen Mao,We build a 3D pose estimation system based mai...,Data Preparation
4,4,https://github.com/JimmySuen/integral-human-pose,Allen Mao,The Integral Regression is also known as soft-...,Data Preparation
...,...,...,...,...,...
399,399,https://github.com/simbody/simbody,Yidan Zhang,"Simbody is a high-performance, open-source too...",Data Visualization
400,400,https://github.com/cyverse/atmosphere,Yidan Zhang,Atmosphere addresses the growing needs for hig...,Data Preparation
401,401,https://github.com/darwinlau/CASPR,Yidan Zhang,The Cable-robot Analysis and Simulation Platfo...,Data Analysis
402,402,https://github.com/microsoft/tensorwatch,Yidan Zhang,TensorWatch is a debugging and visualization t...,Data Visualization


 ## Distribution

In [160]:
count = description_corpura.groupby('label').count()
count

,Unnamed: 0,URL,contributor,excerpt
label,,,,
Data Analysis,117,117,117,117
Data Preparation,75,75,75,75
Data Visualization,139,139,139,139


In [161]:
for tag in func_tag_dic:
    func_corpura.replace(tag,func_tag_dic[tag], inplace=True)
import nltk
from nltk.corpus import stopwords
#Encounter some errors when we downloaded treebank, so use ssl
import ssl
try:
    _create_unverified_https_context=ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('punkt')
#----------Download End-------------------

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd
from sklearn.metrics import precision_score,recall_score,roc_curve, auc, precision_recall_curve, average_precision_score
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
stop_words = stopwords.words('english')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [162]:
X_func, Y_func = description_corpura["excerpt"], description_corpura["label"]

## CountVectorizer + LogisticRegression

In [164]:
pipeline = make_pipeline(CountVectorizer(), LogisticRegression(penalty='l2',multi_class='multinomial',solver = 'lbfgs'))
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)

def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score

pipeline.fit(X_train, Y_train)
y_pred_class = pipeline.predict(X_test)
y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')
#print(y_pred_class)

                    precision    recall  f1-score   support

     Data Analysis       0.61      0.41      0.49        34
  Data Preparation       0.83      0.23      0.36        22
Data Visualization       0.44      0.89      0.59        27

          accuracy                           0.52        83
         macro avg       0.63      0.51      0.48        83
      weighted avg       0.61      0.52      0.49        83

null accuracy: 40.96%
accuracy score: 51.81%
model is 10.84% more accurate than null accuracy


accuracy score: 51.81%
Precision score : 0.6288244766505636
Recall score  : 0.5093087740146564




In [165]:

cv = StratifiedKFold(n_splits = 5, shuffle=True)

cv_results = cross_validate(pipeline, X_func, Y_func, cv=5,scoring='accuracy')
print(cv_results['test_score'])
print('Mean accuracy of CV:',cv_results['test_score'].mean())

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
    
i = 0
print('Description ROC')

[0.37313433 0.58208955 0.5        0.42424242 0.46153846]
Mean accuracy of CV: 0.4682009532755802
Description ROC


## 2) TfidfVectorizer + LogisticRegression


In [178]:
pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression(solver='liblinear'))
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)


def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score


pipeline.fit(X_train, Y_train)
y_pred_class = pipeline.predict(X_test)
y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')
#print(y_pred_class)

                    precision    recall  f1-score   support

     Data Analysis       0.66      0.76      0.70        25
  Data Preparation       1.00      0.18      0.30        17
Data Visualization       0.67      0.83      0.74        41

          accuracy                           0.67        83
         macro avg       0.77      0.59      0.58        83
      weighted avg       0.73      0.67      0.64        83

null accuracy: 49.40%
accuracy score: 67.47%
model is 18.07% more accurate than null accuracy


accuracy score: 67.47%
Precision score : 0.7739463601532567
Recall score  : 0.5885796269727402




/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


## 3) TfidfVectorizer + NaiveBayes¶


In [173]:
pipeline = make_pipeline(TfidfVectorizer(), MultinomialNB())
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)


def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score


pipeline.fit(X_train, Y_train)

y_pred_class = pipeline.predict(X_test)
y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')


                    precision    recall  f1-score   support

     Data Analysis       0.78      0.39      0.52        36
  Data Preparation       1.00      0.28      0.43        18
Data Visualization       0.47      0.97      0.63        29

          accuracy                           0.57        83
         macro avg       0.75      0.54      0.53        83
      weighted avg       0.72      0.57      0.54        83

null accuracy: 43.37%
accuracy score: 56.63%
model is 13.25% more accurate than null accuracy


accuracy score: 56.63%
Precision score : 0.7481481481481481
Recall score  : 0.5440613026819924




### Word Level TF-IDF + NaiveBayes

In [192]:
train_x, valid_x, train_y,valid_y = train_test_split(X_func, Y_func)
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=50)
tfidf_vect.fit(description_corpura["excerpt"])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy_score(predictions, valid_y)

accuracy = train_model(MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print (accuracy)


0.5180722891566265


## 4) TfidfVectorizer + Perceptron¶


In [177]:

pipeline = make_pipeline(TfidfVectorizer(), Perceptron())
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)


def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score


pipeline.fit(X_train, Y_train)

y_pred_class = pipeline.predict(X_test)
#y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')

                    precision    recall  f1-score   support

     Data Analysis       0.50      0.44      0.47        25
  Data Preparation       0.80      0.40      0.53        20
Data Visualization       0.61      0.82      0.70        38

          accuracy                           0.60        83
         macro avg       0.64      0.55      0.57        83
      weighted avg       0.62      0.60      0.59        83

null accuracy: 45.78%
accuracy score: 60.24%
model is 14.46% more accurate than null accuracy


accuracy score: 60.24%
Precision score : 0.6359477124183006
Recall score  : 0.5519298245614035




## TfidfVectorizer + K neighboors

In [194]:
from sklearn.neighbors import KNeighborsClassifier

pipeline = make_pipeline(TfidfVectorizer(), KNeighborsClassifier(n_neighbors=2))
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)


def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score


pipeline.fit(X_train, Y_train)

y_pred_class = pipeline.predict(X_test)
#y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')


                    precision    recall  f1-score   support

     Data Analysis       0.61      0.83      0.70        30
  Data Preparation       0.44      0.57      0.50        14
Data Visualization       0.79      0.49      0.60        39

          accuracy                           0.63        83
         macro avg       0.62      0.63      0.60        83
      weighted avg       0.67      0.63      0.62        83

null accuracy: 46.99%
accuracy score: 62.65%
model is 15.66% more accurate than null accuracy


accuracy score: 62.65%
Precision score : 0.6152890695573622
Recall score  : 0.6306471306471306




In [115]:
from sklearn.ensemble import RandomForestClassifier
pipeline = make_pipeline(TfidfVectorizer(), RandomForestClassifier())
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)


def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score


pipeline.fit(X_train, Y_train)

y_pred_class = pipeline.predict(X_test)
#y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')


                    precision    recall  f1-score   support

     Data Analysis       0.48      0.62      0.54        26
  Data Preparation       1.00      0.26      0.42        19
Data Visualization       0.58      0.68      0.63        38

          accuracy                           0.57        83
         macro avg       0.69      0.52      0.53        83
      weighted avg       0.65      0.57      0.55        83

null accuracy: 45.78%
accuracy score: 56.63%
model is 10.84% more accurate than null accuracy


accuracy score: 56.63%
Precision score : 0.6875420875420876
Recall score  : 0.5209176788124156




/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


### remove stop words

In [121]:

def remove_stopwords(x):
    tokens = nltk.word_tokenize(x)
    tokens=[i for i in tokens if i not in stop_words]
    res=' '.join(tokens)
    
    return res

#del ext_desc['Unnamed: 0']
description_corpura['excerpt'] = description_corpura['excerpt'].apply(remove_stopwords)
X_func, Y_func = description_corpura["excerpt"], description_corpura["label"]

In [123]:

pipeline = make_pipeline(TfidfVectorizer(), Perceptron())
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)


def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score


pipeline.fit(X_train, Y_train)

y_pred_class = pipeline.predict(X_test)
#y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')

                    precision    recall  f1-score   support

     Data Analysis       0.58      0.53      0.55        34
  Data Preparation       0.47      0.47      0.47        17
Data Visualization       0.51      0.56      0.54        32

          accuracy                           0.53        83
         macro avg       0.52      0.52      0.52        83
      weighted avg       0.53      0.53      0.53        83

null accuracy: 40.96%
accuracy score: 53.01%
model is 12.05% more accurate than null accuracy


accuracy score: 53.01%
Precision score : 0.5218397036233848
Recall score  : 0.5208333333333334




## NLP

In [127]:
import string 

In [128]:
description_corpura['char_count'] = description_corpura['excerpt'].apply(len)
description_corpura['word_count'] = description_corpura['excerpt'].apply(lambda x: len(x.split()))
description_corpura['word_density'] = description_corpura['char_count'] / (description_corpura['word_count']+1)
description_corpura['punctuation_count'] = description_corpura['excerpt'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
description_corpura['title_word_count'] = description_corpura['excerpt'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
description_corpura['upper_case_word_count'] = description_corpura['excerpt'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [130]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

description_corpura['noun_count'] = description_corpura['excerpt'].apply(lambda x: check_pos_tag(x, 'noun'))
description_corpura['verb_count'] = description_corpura['excerpt'].apply(lambda x: check_pos_tag(x, 'verb'))
description_corpura['adj_count'] = description_corpura['excerpt'].apply(lambda x: check_pos_tag(x, 'adj'))
description_corpura['adv_count'] = description_corpura['excerpt'].apply(lambda x: check_pos_tag(x, 'adv'))
description_corpura['pron_count'] = description_corpura['excerpt'].apply(lambda x: check_pos_tag(x, 'pron'))


In [131]:
description_corpura

,Unnamed: 0,URL,contributor,excerpt,label,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,noun_count,verb_count,adj_count,adv_count,pron_count
0,0,https://github.com/GoogleChrome/puppeteer,Allen Mao,Puppeteer Node library provides high-level API...,Data Preparation,179,26,6.629630,7,8,1,0,0,0,0,0
1,1,https://github.com/JimmySuen/integral-human-pose,Allen Mao,The major contributors repository include Xiao...,Data Preparation,105,20,5.000000,5,11,0,0,0,0,0,0
2,2,https://github.com/JimmySuen/integral-human-pose,Allen Mao,Integral Regression initially described ECCV 2...,Data Preparation,70,12,5.384615,4,3,1,0,0,0,0,0
3,3,https://github.com/JimmySuen/integral-human-pose,Allen Mao,We build 3D pose estimation system based mainl...,Data Preparation,333,50,6.529412,11,17,5,0,0,0,0,0
4,4,https://github.com/JimmySuen/integral-human-pose,Allen Mao,The Integral Regression also known soft-argmax...,Data Preparation,168,27,6.000000,7,6,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,399,https://github.com/simbody/simbody,Yidan Zhang,"Simbody high-performance , open-source toolkit...",Data Visualization,344,43,7.818182,13,1,0,0,0,0,0,0
400,400,https://github.com/cyverse/atmosphere,Yidan Zhang,Atmosphere addresses growing needs highly conf...,Data Preparation,131,15,8.187500,1,1,0,0,0,0,0,0
401,401,https://github.com/darwinlau/CASPR,Yidan Zhang,The Cable-robot Analysis Simulation Platform R...,Data Analysis,179,22,7.782609,6,5,2,0,0,0,0,0
402,402,https://github.com/microsoft/tensorwatch,Yidan Zhang,TensorWatch debugging visualization tool desig...,Data Visualization,122,15,7.625000,2,2,0,0,0,0,0,0


In [132]:
pipeline = make_pipeline(CountVectorizer(),naive_bayes.MultinomialNB())
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)

def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score

pipeline.fit(X_train, Y_train)
y_pred_class = pipeline.predict(X_test)
y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')
#print(y_pred_class)

NameError: name 'naive_bayes' is not defined